In [1]:
%matplotlib inline
import os
import sys
import gc
import time
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pygame
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.model_wrapper import ModelWrapper
from src.learning.training.collector import Collector
from src.learning.training.training_file_reader import TrainingFileReader
from src.utilities.transformer import Transformer
from notebooks.notebook_commons import read_stored_data_with_labels, read_stored_data_with_shifted_labels, create_memorized_dataset

window_width = 500
window_height = 200

black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
green = (0, 153, 0)
blue = (0, 128, 255)

pygame.init()
font = pygame.font.SysFont('Roboto', 20)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def plot_stuff(title, plot_elems, figsize=(18, 6)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Count')
    
    for plot_elem in plot_elems:
        plt.plot(plot_elem['data'], label=plot_elem['label'], alpha=plot_elem['alpha'])

    plt.grid(axis='y')
    plt.legend(loc='best')
    plt.show()
        
def get_mem_dataset_with_full_video(filename, length, interval):
    mem_frames_np, mem_telemetry_np, mem_diffs_np = create_memorized_dataset(*read_stored_data_with_labels(reader, transformer, filename, collector.steering_columns(), collector.diff_steering_columns()), length, interval)
    view_frames_np = reader.read_video(filename + '.avi')[((length - 1) * interval):]
    
    assert view_frames_np.shape[0] == mem_frames_np.shape[0], "Frames length mismatch!"
    assert mem_frames_np.shape[0] == mem_telemetry_np.shape[0] == mem_diffs_np.shape[0], "Mem lengths differ!"
    return mem_frames_np, mem_telemetry_np, mem_diffs_np, view_frames_np

def get_shifted_mem_dataset_with_full_video(filename, length, interval):
    mem_frames_np, mem_telemetry_np, mem_diffs_np = create_memorized_dataset(*read_stored_data_with_shifted_labels(reader, transformer.cut_wide_and_normalize_video_shifted, filename, collector.numeric_columns(), collector.steering_column()), length, interval)
    view_frames_np = reader.read_video(filename + '.avi')[((length - 1) * interval):-1]
    
    assert view_frames_np.shape[0] == mem_frames_np.shape[0], "Frames length mismatch!"
    assert mem_frames_np.shape[0] == mem_telemetry_np.shape[0] == mem_diffs_np.shape[0], "Mem lengths differ!"
    return mem_frames_np, mem_telemetry_np, mem_diffs_np, view_frames_np

def validate_partial(memory, i):
    memory_string = 'n{}_m{}'.format(*memory)
    path = '../../training/' + memory_string + '/'

    mem_frame = np.load(path + GenFiles.frame.format(memory_string, i)) * 255
    return mem_frame.astype(np.uint8)[:,:,:3]

In [3]:
config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/validation_laps/')
collector = Collector()

validation_laps = ['2020_02_28_lap_20']
validation_lap = validation_laps[0]
memory_variants = [(1, 1), (4, 1), (4, 4), (8, 1), (8, 4), (12, 1)]
memory = memory_variants[1]

transformer = Transformer(config, memory_tuple=memory)

mem_frames, mem_telems, mem_diffs, display_frames = get_shifted_mem_dataset_with_full_video(validation_lap, *memory)
print(mem_frames.shape)
print(mem_telems.shape)

(2929, 50, 180, 12)
(2929, 16)


In [4]:
from tensorflow.keras.models import load_model
#multi = load_model('../../training/models/model_n{}_m{}_6.h5'.format(*memory))
multi = ModelWrapper(config, memory_tuple=memory, model_num=1)

Loaded model_n1_m1_1.h5
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 50, 180, 3)]      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 25, 90, 24)        1824      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 45, 36)        21636     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 23, 48)         43248     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 7, 23, 64)         27712     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 7, 23, 64)         36928     
_________________________________________________________________
flatten_2 (Flatten)          (None,

In [5]:
pred_steer = []
actual_steer = []

pygame.display.init()
pygame.display.set_caption("Prediction viewer")
screen = pygame.display.set_mode((window_width, window_height))

try:
    pred_smoothing = []
    for i in range(0, display_frames.shape[0]):
        frame = display_frames[i]
        screen.fill(black)
        
        converted_telem = mem_telems[i][np.newaxis, :]
        converted_frame = mem_frames[i][np.newaxis, :]
        
        # gear, steering, throttle, braking
        #predictions = multi.predict([converted_frame, converted_telem])[0]
        #predicted_steering = predictions[0]
        predicted_steering = multi.predict(mem_frames[i], mem_telems[i]).to_dict()['d_steering']
        
        pred_steering_str = "P: {0:.4f}".format(predicted_steering)
        if predicted_steering >= 0.0:
            prediction_text = font.render(pred_steering_str, True, green)
        else:
            prediction_text = font.render(pred_steering_str, True, blue)
        
        actual_steering = mem_telems[i, 1]
        #actual_steering = mem_telems[i, 0]
        actual_steering_str = "T: {0:.4f}".format(actual_steering)
        if actual_steering >= 0.0:
            actual_text = font.render(actual_steering_str, True, green)
        else:
            actual_text = font.render(actual_steering_str, True, blue)
        
        pred_steer.append(predicted_steering)
        actual_steer.append(actual_steering)
        
        delta = actual_steering - predicted_steering
        delta_str = "D: {0:.4f}".format(delta)
        delta_text = font.render(delta_str, True, white)
        
        #frame = mem_frames[i][:,:,:3]
        #frame = validate_partial(memory, i)
        frame = np.rot90(frame)
        surface = pygame.surfarray.make_surface(frame)
        
        x = (window_width - frame.shape[0]) // 2
        y = 0
        
        #time.sleep(0.05)
        screen.blit(prediction_text, (200, window_height - 75))
        screen.blit(actual_text, (200, window_height - 45))
        screen.blit(delta_text, (200, window_height - 15))
        screen.blit(surface, (x, y))
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    sys.exit(0)
    pygame.display.quit()
except (KeyboardInterrupt, SystemExit, Exception) as ex:
    print(ex)
    pygame.display.quit()

Error when checking input: expected input_3 to have shape (50, 180, 3) but got array with shape (50, 180, 12)


In [ ]:
plotto = []
plotto.append({'data': actual_steer, 'label': 'actual', 'alpha': 0.9})
plotto.append({'data': pred_steer, 'label': 'predicted', 'alpha': 0.9})

plot_stuff('steering comparison', plotto)

drive_data = reader.read_specific_telemetry_columns(validation_lap + '.csv', ['cs', 'p_steering'])

plotto_2 = []
plotto_2.append({'data': drive_data.to_numpy()[:, 0], 'label': 'actual', 'alpha': 0.9})
plotto_2.append({'data': drive_data.to_numpy()[:, 1], 'label': 'predicted', 'alpha': 0.9})

plot_stuff('live steering comparison', plotto_2)

In [ ]:
df = reader.read_specific_telemetry_columns('2020_02_27_i1.csv', ['c', 'conn_time', 'p_start', 'p_end'])
df.drop(df.tail(1).index,inplace=True)
data = df.to_numpy()

plot_data = []
plot_data.append({'data': data[10:, 1]- data[10:, 0], 'label': 'conn diff millis', 'alpha': 0.7})
plot_data.append({'data': data[10:, 3]- data[10:, 2], 'label': 'p diff millis', 'alpha': 0.7})
plot_stuff('connection and predict time elapsed', plot_data)

plot_data = []
plot_data.append({'data': data[:, 2]- data[:, 1], 'label': 'zmq diff millis', 'alpha': 0.7})
plot_stuff('time diff between connection and predict', plot_data)

t = np.mean(data[1:, 3]- data[1:, 2])
print(t)
print('FPS: {}'.format(1/(t / 1000)))

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices())

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
a = {'a': 2}
b = a
b['b'] = 3

print(a)
print(b)

from PIL import Image

frame = mem_frames[2250][:,:,:3]
Image.fromarray((frame * 255).astype(np.uint8)).show()

from datetime import datetime
print(datetime.now().timestamp())